In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Read the CSV file
df = pd.read_csv("profile_dataNewest.csv")

# Main function to create GCN and perform forward pass
def gcn_main():
    # Initialize NetworkX graph and dictionaries
    G = nx.Graph()
    label = {}
    likes = {}

    # Create nodes in the graph
    for i, (user, category) in enumerate(zip(df['first_name'], df['categories'])):
        label[i] = user  # Use index as node label
        G.add_node(i)
        likes[user] = category

    # Create edges based on common categories
    for u in range(len(label)):
        for v in range(len(label)):
            if u != v:
                categories1 = set(likes[label[u]].split(','))
                categories2 = set(likes[label[v]].split(','))
                common_categories = categories1.intersection(categories2)
                if common_categories:
                    G.add_edge(u, v)

    # Check if the graph has any edges
    if G.number_of_edges() == 0:
        print("Error: The graph has no edges.")
        return

    # Convert edges to a list of tuples
    edges = list(G.edges())

    # Create a sample data with node features (x) and edge indices
    x = torch.randn(len(label), 16)  # Random node features
    edge_index = torch.tensor(edges).t().contiguous()  # Edge indices
    data = Data(x=x, edge_index=edge_index)

    # Instantiate GCN model
    gcn_model = GCNModel(input_dim=16, hidden_dim=32, output_dim=2)

    # Forward pass
    gcn_output = gcn_model(data)

    # Visualize the graph
    visualize_graph(G, label)

    return gcn_output

# GCN Model
class GCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return F.log_softmax(x, dim=1)

# Function to visualize the graph
def visualize_graph(G, label):
    pos = nx.shell_layout(G)
    nx.draw(G, pos, labels=label, with_labels=True, node_size=500)
    plt.draw()
    plt.show()

if __name__ == "__main__":
    gcn_main()
